In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [43]:
from urllib.request import FancyURLopener

# url = "http://www.hubertiming.com/results/2017GPTR10K"
url = "https://www.sherdog.com/fighter/Jon-Jones-27944"
# html = urlopen(url)
class MyOpener(FancyURLopener):
    version = 'My new User-Agent'

In [44]:
MyOpener.version
class MyOpener(FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11'

In [45]:
myopener = MyOpener()
page = myopener.open(url)
soup = BeautifulSoup(page, 'lxml')

C:\Users\Vincent.Cheng\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


In [46]:
text = soup.get_text()
text

'\n\nJon "Bones" Jones MMA Stats, Pictures, News, Videos, Biography - Sherdog.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    //<!--\n\n    $(function() {\n\t    var slideVideoPanelPage = 0;\n\t    var slideVideoPanelLimit = parseInt($("#sd_video_list div.item").length / 3) + ($("#sd_video_list div.item").length % 3 > 0 ? 1 : 0);\n\t    function slideVideoPanel(page) {\n\t        var init = page * 3;\n\t        var end = init + 3; \n\t        $("#sd_video_list div.item").fadeOut().filter(function(index){\n\t            return index >= init && index < end;\n            }).fadeIn();\n            slideVideoPanelPage = page;\n\t    }\n\t    slideVideoPanel(slideVideoPanelPage);\n\t    $(".sd_video_list .footer div").click(function(){\n            if ($(this).hasClass("left")) {\n                if (slideVideoPanelPage > 0) {\n                    slideVideoPanel(slideVideoPanelPage - 1);\n                }\n            } else {\n                if (slideVideoPanelPage < (slideVideoPanelL

In [47]:
soup.find_all('tr')

[<tr class="table_head">
 <td class="col_one">Date</td>
 <td class="col_two">VS. Fighter</td>
 <td class="col_three">Event</td>
 <td class="col_Four">Location</td>
 </tr>, <tr>
 <td class="event-upcoming" colspan="4">
 <section>
 <div class="module" itemprop="performerIn" itemscope="" itemtype="http://schema.org/Event">
 <div class="module_header">
 <h2 class="text_center" itemprop="name">UFC 239</h2>
 </div>
 <div class="content event">
 <div class="date_location text_center">
 <meta content="2019-07-06T00:00:00-07:00" itemprop="startDate"/>
 <h4>July 06, 2019 / <em itemprop="location" itemscope="" itemtype="http://schema.org/Place"><span itemprop="name">T-Mobile Arena</span>, <span itemprop="address">Las Vegas, United States, United States</span>
 </em></h4>
 </div>
 <div class="fight">
 <span class="versus">Versus</span>
 <div class="fighter left_side" itemprop="performer" itemscope="" itemtype="http://schema.org/Person">
 <a href="/fighter/Jon-Jones-27944" itemprop="url"><img alt="

In [48]:
rows = soup.find_all('tr')

for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td height="37" width="37"><img data-src="{image}" width="32"/></td>, <td>{name} - {title}</td>]


bs4.element.ResultSet

In [49]:
str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)

[
Date
VS. Fighter
Event
Location
, 




UFC 239




July 06, 2019 / T-Mobile Arena, Las Vegas, United States, United States



Versus


Jon Jones
24 - 1 - 0
(Win - Loss - Draw)



Thiago Santos
21 - 6 - 0
(Win - Loss - Draw)



See entire fight card





, 
Result
Fighter
Event
Method/Referee
R
Time
, 
win
Anthony Smith
UFC 235 - Jones vs. SmithMar / 02 / 2019
Decision (Unanimous)Herb Dean
5
5:00
, 
win
Alexander Gustafsson
UFC 232 - Jones vs. Gustafsson 2Dec / 29 / 2018
KO (Punches)Mike Beltran
3
2:02
, 
NC
Daniel Cormier
UFC 214 - Cormier vs. Jones 2Jul / 29 / 2017
No Contest (Overturned by CSAC)John McCarthy
3
3:01
, 
win
Ovince St. Preux
UFC 197 - Jones vs. St. PreuxApr / 23 / 2016
Decision (Unanimous)Herb Dean
5
5:00
, 
win
Daniel Cormier
UFC 182 - Jones vs. CormierJan / 03 / 2015
Decision (Unanimous)Herb Dean
5
5:00
, 
win
Glover Teixeira
UFC 172 - Jones vs. TeixeiraApr / 26 / 2014
Decision (Unanimous)Dan Miragliotta
5
5:00
, 
win
Alexander Gustafsson
UFC 165 - Jones vs. Gustafs

In [50]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(list_rows)
type(clean2)

['[Date, VS. Fighter, Event, Location]', '[\n\n\n\nUFC 239\n\n\n\n\nJuly 06, 2019 / T-Mobile Arena, Las Vegas, United States, United States\n\n\n\nVersus\n\n\nJon Jones\n24 - 1 - 0\n(Win - Loss - Draw)\n\n\n\nThiago Santos\n21 - 6 - 0\n(Win - Loss - Draw)\n\n\n\nSee entire fight card\n\n\n\n\n, \n]', '[Result, Fighter, Event, Method/Referee, R, Time]', '[win, Anthony Smith, UFC 235 - Jones vs. SmithMar / 02 / 2019, Decision (Unanimous)Herb Dean, 5, 5:00]', '[win, Alexander Gustafsson, UFC 232 - Jones vs. Gustafsson 2Dec / 29 / 2018, KO (Punches)Mike Beltran, 3, 2:02]', '[NC, Daniel Cormier, UFC 214 - Cormier vs. Jones 2Jul / 29 / 2017, No Contest (Overturned by CSAC)John McCarthy, 3, 3:01]', '[win, Ovince St. Preux, UFC 197 - Jones vs. St. PreuxApr / 23 / 2016, Decision (Unanimous)Herb Dean, 5, 5:00]', '[win, Daniel Cormier, UFC 182 - Jones vs. CormierJan / 03 / 2015, Decision (Unanimous)Herb Dean, 5, 5:00]', '[win, Glover Teixeira, UFC 172 - Jones vs. TeixeiraApr / 26 / 2014, Decision

str

In [61]:
df = pd.DataFrame(list_rows)
# df.columns = df.iloc[0]
# df.reindex(df.index.drop(0))

df.head(10)

,0
0,"[Date, VS. Fighter, Event, Location]"
1,"[\n\n\n\nUFC 239\n\n\n\n\nJuly 06, 2019 / T-Mo..."
2,"[Result, Fighter, Event, Method/Referee, R, Time]"
3,"[win, Anthony Smith, UFC 235 - Jones vs. Smith..."
4,"[win, Alexander Gustafsson, UFC 232 - Jones vs..."
5,"[NC, Daniel Cormier, UFC 214 - Cormier vs. Jon..."
6,"[win, Ovince St. Preux, UFC 197 - Jones vs. St..."
7,"[win, Daniel Cormier, UFC 182 - Jones vs. Corm..."
8,"[win, Glover Teixeira, UFC 172 - Jones vs. Tei..."
9,"[win, Alexander Gustafsson, UFC 165 - Jones vs..."


In [76]:
df1 = df[0].str.split(',', expand=True)
df1[0] = df1[0].str.strip('[')
df1[5] = df1[5].str.strip(']')
df1.columns = df1.iloc[2]
# df1 = df1.reindex(df1.index.drop(2))
df1.head(10)

2,Result,Fighter,Event,Method/Referee,R,Time
0,Date,VS. Fighter,Event,Location],None,None
1,\n\n\n\nUFC 239\n\n\n\n\nJuly 06,2019 / T-Mobile Arena,Las Vegas,United States,United States\n\n\n\nVersus\n\n\nJon Jones\n2...,\n
2,Result,Fighter,Event,Method/Referee,R,Time
3,win,Anthony Smith,UFC 235 - Jones vs. SmithMar / 02 / 2019,Decision (Unanimous)Herb Dean,5,5:00
4,win,Alexander Gustafsson,UFC 232 - Jones vs. Gustafsson 2Dec / 29 / 2018,KO (Punches)Mike Beltran,3,2:02
5,NC,Daniel Cormier,UFC 214 - Cormier vs. Jones 2Jul / 29 / 2017,No Contest (Overturned by CSAC)John McCarthy,3,3:01
6,win,Ovince St. Preux,UFC 197 - Jones vs. St. PreuxApr / 23 / 2016,Decision (Unanimous)Herb Dean,5,5:00
7,win,Daniel Cormier,UFC 182 - Jones vs. CormierJan / 03 / 2015,Decision (Unanimous)Herb Dean,5,5:00
8,win,Glover Teixeira,UFC 172 - Jones vs. TeixeiraApr / 26 / 2014,Decision (Unanimous)Dan Miragliotta,5,5:00
9,win,Alexander Gustafsson,UFC 165 - Jones vs. GustafssonSep / 21 / 2013,Decision (Unanimous)John McCarthy,5,5:00


In [106]:
cols=list(df2.columns)


df2 = df1.drop([0,1,2,29,30,31],axis = 0)
df2[cols[3]] = df2[cols[3]].str.replace(')', '))')
df2[cols[3]].str.rsplit(')', expand=True, n=1)

,0,1
3,Decision (Unanimous),Herb Dean
4,KO (Punches),Mike Beltran
5,No Contest (Overturned by CSAC),John McCarthy
6,Decision (Unanimous),Herb Dean
7,Decision (Unanimous),Herb Dean
8,Decision (Unanimous),Dan Miragliotta
9,Decision (Unanimous),John McCarthy
10,TKO (Elbows and Punches),Keith Peterson
11,Submission (Keylock),John McCarthy
12,Decision (Unanimous),Herb Dean


3                         Decision (Unanimous)Herb Dean
4                              KO (Punches)Mike Beltran
5          No Contest (Overturned by CSAC)John McCarthy
6                         Decision (Unanimous)Herb Dean
7                         Decision (Unanimous)Herb Dean
8                   Decision (Unanimous)Dan Miragliotta
9                     Decision (Unanimous)John McCarthy
10               TKO (Elbows and Punches)Keith Peterson
11                    Submission (Keylock)John McCarthy
12                        Decision (Unanimous)Herb Dean
13     Technical Submission (Guillotine Choke)John M...
14          Submission (Rear-Naked Choke)Josh Rosenthal
15                     TKO (Punches and Knees)Herb Dean
16               Submission (Guillotine Choke)Herb Dean
17                                TKO (Elbows)Herb Dean
18                    TKO (Elbows and Punches)Herb Dean
19     Disqualification (Illegal Elbows)Steve Mazzag...
20            Submission (Guillotine Choke)Yves 

In [111]:
df2.reset_index(drop = True)

2,Result,Fighter,Event,Method/Referee,R,Time
0,win,Anthony Smith,UFC 235 - Jones vs. SmithMar / 02 / 2019,Decision (Unanimous))Herb Dean,5,5:00
1,win,Alexander Gustafsson,UFC 232 - Jones vs. Gustafsson 2Dec / 29 / 2018,KO (Punches))Mike Beltran,3,2:02
2,NC,Daniel Cormier,UFC 214 - Cormier vs. Jones 2Jul / 29 / 2017,No Contest (Overturned by CSAC))John McCarthy,3,3:01
3,win,Ovince St. Preux,UFC 197 - Jones vs. St. PreuxApr / 23 / 2016,Decision (Unanimous))Herb Dean,5,5:00
4,win,Daniel Cormier,UFC 182 - Jones vs. CormierJan / 03 / 2015,Decision (Unanimous))Herb Dean,5,5:00
5,win,Glover Teixeira,UFC 172 - Jones vs. TeixeiraApr / 26 / 2014,Decision (Unanimous))Dan Miragliotta,5,5:00
6,win,Alexander Gustafsson,UFC 165 - Jones vs. GustafssonSep / 21 / 2013,Decision (Unanimous))John McCarthy,5,5:00
7,win,Chael Sonnen,UFC 159 - Jones vs. SonnenApr / 27 / 2013,TKO (Elbows and Punches))Keith Peterson,1,4:33
8,win,Vitor Belfort,UFC 152 - Jones vs. BelfortSep / 22 / 2012,Submission (Keylock))John McCarthy,4,0:54
9,win,Rashad Evans,UFC 145 - Jones vs. EvansApr / 21 / 2012,Decision (Unanimous))Herb Dean,5,5:00
